### 1.SSE different format
#SSE native, 上交所取，have b1n-b10n, a1n-a10n, in total 194 columns (csv, 2019.7z)
#SSE XTP, after 20190101 (2019旧和2020), doesn't have numofOrders, in total 174 columns (csv/.7z)

### 2.SSE data position
#G: 2016\0104-1230
#G: 2017\0103-1229
#G: 2018\20180102-20181228
#G: 2019\20190102-20190702
#L: 2019\20190703-20191231
#L: 2020 data\SH\20200102-20200508

### 3.Change columns format

In [3]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time
pd.set_option("max_columns", 200)

    
colnames=['StockID', 'DateTime', 'prevClose', 'open','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','b1','b2','b3','b4','b5','b6','b7','b8','b9','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b20',
          'b21','b22','b23','b24','b25','b26','b27','b28','b29','b30','b31','b32','b33','b34','b35','b36','b37','b38','b39','b40','b41','b42','b43',
          'b44','b45','b46','b47','b48','b49','b50','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14',
         'a15','a16','a17','a18','a19','a20','a21','a22','a23','a24','a25','a26','a27','a28','a29','a30','a31','a32','a33','a34','a35','a36','a37','a38',
         'a39','a40','a41','a42','a43','a44','a45','a46','a47','a48','a49','a50','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames1=['StockID', 'DateTime', 'prevClose', 'open','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n',
           'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
           'ask7n', 'ask8n', 'ask9n', 'ask10n','ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames2=['SecurityID', 'DateTime', 'Price', 'Qty', 'LeaveQty', "Side"]

startTm = datetime.datetime.now()
year = "2018"
df = []
bad = []

readPath = 'G:\\' + year + '\\***' 
dataPathLs = np.array(glob.glob(readPath))

for data in dataPathLs[:1]:
    path = np.array(glob.glob(data +'\\***'))[[os.path.basename(i).split('.')[0] == 'Snapshot' \
                                                               for i in np.array(glob.glob(data +'\\***'))]][0]
    
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue

    else:
        try:
            snapshot = pd.read_csv(path, header=None)
            assert(snapshot.shape[1] == len(colnames))
            print("XTP lv2 data")
            snapshot.columns = colnames
            snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
            snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)
        except:
            try1 = pd.read_csv(path, nrows=1, header=None)
            if try1.shape[1] == 2:
                print("native lv2 data")
                snapshot = pd.read_csv(path, header=None, names=colnames1).iloc[1:, :]
                snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
                snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)                
        print("csv is loaded")
        
        snapshot["StockID"] = snapshot["StockID"].astype(int)
        snapshot = snapshot[(snapshot["StockID"] >= 600000) & (snapshot["StockID"] < 700000)].reset_index(drop=True)
        snapshot = snapshot.rename(columns={"StockID": "ID"})
        snapshot["date"] = int(snapshot["DateTime"].iloc[0]//1000000)
        snapshot["clockAtArrival"] = snapshot["DateTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S').timestamp()*1e6))
        # snapshot["clockAtArrival"] = pd.to_datetime(snapshot["DateTime"].astype(str), format = '%Y%m%d%H%M%S').apply(lambda x: int(x.timestamp()*1e6))
        snapshot['ID'] = 1000000 + snapshot["ID"]
        snapshot["ordering"] = snapshot.groupby("ID").cumcount()
        snapshot["ordering"] = snapshot["ordering"] + 1
        snapshot = snapshot.rename(columns={"NumTrades": "cum_tradesCnt", "HighPx":"high", "LowPx":"low", "TotalBidQty":"totalBidQuantity",
                                   "TotalOfferQty":"totalAskQuantity", "WeightedAvgBidPx":"vwapBid", "WeightedAvgOfferPx":"vwapAsk",
                                   "TotalBidNumber":"totalBidOrders", "TotalOfferNumber":"totalAskOrders", "NumBidOrders":"totalBidLevels",
                                   "NumOfferOrders":"totalAskLevels", "WithdrawBuyNumber":"cum_canceledBuyOrders",
                                   "WithdrawBuyAmount":"cum_canceledBuyVolume", "WithdrawBuyMoney":"cum_canceledBuyAmount",
                                   "WithdrawSellNumber":"cum_canceledSellOrders", "WithdrawSellAmount":"cum_canceledSellVolume",
                                   "WithdrawSellMoney":"cum_canceledSellAmount", "BidTradeMaxDuration":"bidTradeMaxDuration",
                                    "OfferTradeMaxDuration":"askTradeMaxDuration"})
        
        try1 = pd.read_csv(data +'\\Auction.csv', nrows=1, header=None)
        if try1.shape[1] == 2:
            print("native auction data")
            aucData = pd.read_csv(data +'\\Auction.csv', header=None, names=colnames2).iloc[1:, :]
        aucData['bid1p'] = aucData['Price']
        aucData['ask1p'] = aucData['Price']
        aucData['bid1q'] = aucData['Qty']
        aucData['ask1q'] = aucData['Qty']
        aucData['bid2q'] = np.where(aucData['Side'] == 1, aucData['LeaveQty'], 0)
        aucData['ask2q'] = np.where(aucData['Side'] == 2, aucData['LeaveQty'], 0)
        aucData["SecurityID"] = aucData["SecurityID"].astype(int)
        aucData = aucData[(aucData["SecurityID"] >= 600000) & (aucData["SecurityID"] < 700000)].reset_index(drop=True)
        aucData["ID"] = aucData["SecurityID"] + 1000000
        aucData["DateTime"] = aucData["DateTime"].apply(lambda x: int(x))
        aucData["date"] = int(aucData["DateTime"].iloc[0]//1000000)
        aucData['time'] = ((aucData["DateTime"] - int(aucData["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)
        aucData["clockAtArrival"] = aucData["DateTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S').timestamp()*1e6))
        aucData = aucData[['date', 'time', 'clockAtArrival', 'ID', 'bid1p', 'ask1p', 'bid2q', 'bid1q', 'ask1q', 'ask2q']].reset_index(drop=True)
        test1 = aucData.groupby("ID")["time"].max().reset_index()
        test2 = snapshot.groupby("ID")["time"].min().reset_index()
        test = pd.merge(test1, test2, on="ID")
        assert(test[test["time_x"] >= test["time_y"]].shape[0]==0)
        
        snapshot = pd.concat([aucData, snapshot]).reset_index(drop=True)
        snapshot = snapshot.fillna(0)
        
        for col in ["ID", "cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q', 'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', "totalBidQuantity", "totalAskQuantity", "totalBidOrders",
        'totalAskOrders', 'totalBidLevels', 'totalAskLevels', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume', 'cum_canceledSellOrders',
        'cum_canceledSellVolume', 'cancelBuyOrders', 'cancelBuyVolume', 'cancelSellOrders', 'cancelSellVolume', "ordering",
        'bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n', 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
        'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bidTradeMaxDuration', 'askTradeMaxDuration']:
            snapshot[col] = snapshot[col].astype(np.int64)
        
        snapshot["ordering"] = snapshot.groupby("ID").cumcount()
        snapshot["ordering"] = snapshot["ordering"] + 1
        
        for cols in ["prevClose", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'vwapBid', "vwapAsk"]:
            snapshot[cols] = (snapshot[cols] * 10000).round(0).astype(np.int64)
        
        for cols in ["cum_amount", "cum_canceledBuyAmount", "cum_canceledSellAmount", "cancelBuyAmount", "cancelSellAmount"]:
            snapshot[cols] = (snapshot[cols] * 10000).round(0).astype(np.int64)
        
        
        display(snapshot.groupby("ID")["time"].min().max())
        
        
        
        snapshot = snapshot[["date", "time", "clockAtArrival", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQuantity", "totalAskQuantity","vwapBid", "vwapAsk",
        "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]]

native lv2 data
csv is loaded
native auction data


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



92507000000

In [4]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time

startTm = datetime.datetime.now()
year = "2018"
df = []
bad = []

readPath = 'G:\\' + year + '\\***\\Auction***' 
len(np.array(glob.glob(readPath)))

245

In [ ]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time
pd.set_option("max_columns", 200)

    
colnames=['StockID', 'DateTime', 'prevClose', 'open','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','b1','b2','b3','b4','b5','b6','b7','b8','b9','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b20',
          'b21','b22','b23','b24','b25','b26','b27','b28','b29','b30','b31','b32','b33','b34','b35','b36','b37','b38','b39','b40','b41','b42','b43',
          'b44','b45','b46','b47','b48','b49','b50','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14',
         'a15','a16','a17','a18','a19','a20','a21','a22','a23','a24','a25','a26','a27','a28','a29','a30','a31','a32','a33','a34','a35','a36','a37','a38',
         'a39','a40','a41','a42','a43','a44','a45','a46','a47','a48','a49','a50','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames1=['StockID', 'DateTime', 'prevClose', 'open','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n',
           'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
           'ask7n', 'ask8n', 'ask9n', 'ask10n','ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames2=['SecurityID', 'DateTime', 'Price', 'Qty', 'LeaveQty', "Side"]

startTm = datetime.datetime.now()
year = "2019"
df = []
bad = []

# readPath = 'G:\\' + year +'\\***' 
readPath = 'F:\\' + year +'\\***'
dataPathLs = np.array(glob.glob(readPath))

for data in dataPathLs[:1]:
    path = np.array(glob.glob(data +'\\***'))[[os.path.basename(i).split('.')[0] == 'Snapshot' \
                                                               for i in np.array(glob.glob(data +'\\***'))]][0]
    
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue

    else:
        try:
            snapshot = pd.read_csv(data + '\\Snapshot.csv', header=None)
            assert(snapshot.shape[1] == len(colnames))
            print("XTP lv2 data")
            snapshot.columns = colnames
            snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
            snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)
        except:
            try1 = pd.read_csv(data + '\\Snapshot.csv', nrows=1, header=None)
            if try1.shape[1] == 2:
                print("native lv2 data")
                snapshot = pd.read_csv(data + '\\Snapshot.csv', header=None, names=colnames1).iloc[1:, :]
                snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
                snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)                
        print("csv is loaded")
        
        snapshot["StockID"] = snapshot["StockID"].astype(int)
        snapshot = snapshot[(snapshot["StockID"] >= 600000) & (snapshot["StockID"] < 700000)].reset_index(drop=True)
        snapshot = snapshot.rename(columns={"StockID": "ID"})
        snapshot["date"] = int(snapshot["DateTime"].iloc[0]//1000000)
        snapshot["clockAtArrival"] = snapshot["DateTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S').timestamp()*1e6))
        # snapshot["clockAtArrival"] = pd.to_datetime(snapshot["DateTime"].astype(str), format = '%Y%m%d%H%M%S').apply(lambda x: int(x.timestamp()*1e6))
        snapshot['ID'] = 1000000 + snapshot["ID"]
        snapshot["ordering"] = snapshot.groupby("ID").cumcount()
        snapshot["ordering"] = snapshot["ordering"] + 1
        snapshot = snapshot.rename(columns={"NumTrades": "cum_tradesCnt", "HighPx":"high", "LowPx":"low", "TotalBidQty":"totalBidQuantity",
                                   "TotalOfferQty":"totalAskQuantity", "WeightedAvgBidPx":"vwapBid", "WeightedAvgOfferPx":"vwapAsk",
                                   "TotalBidNumber":"totalBidOrders", "TotalOfferNumber":"totalAskOrders", "NumBidOrders":"totalBidLevels",
                                   "NumOfferOrders":"totalAskLevels", "WithdrawBuyNumber":"cum_canceledBuyOrders",
                                   "WithdrawBuyAmount":"cum_canceledBuyVolume", "WithdrawBuyMoney":"cum_canceledBuyAmount",
                                   "WithdrawSellNumber":"cum_canceledSellOrders", "WithdrawSellAmount":"cum_canceledSellVolume",
                                   "WithdrawSellMoney":"cum_canceledSellAmount", "BidTradeMaxDuration":"bidTradeMaxDuration",
                                    "OfferTradeMaxDuration":"askTradeMaxDuration"})

        snapshot = snapshot.fillna(0)
        
        for col in ["ID", "cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q', 'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', "totalBidQuantity", "totalAskQuantity", "totalBidOrders",
        'totalAskOrders', 'totalBidLevels', 'totalAskLevels', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume', 'cum_canceledSellOrders',
        'cum_canceledSellVolume', "ordering",
        'bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n', 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
        'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bidTradeMaxDuration', 'askTradeMaxDuration']:
            snapshot[col] = snapshot[col].astype(np.int64)
        
        for cols in ["prevClose", "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'vwapBid', "vwapAsk"]:
            snapshot[cols] = (snapshot[cols] * 10000).round(0).astype(np.int64)
        
        for cols in ["cum_amount", "cum_canceledBuyAmount", "cum_canceledSellAmount"]:
            snapshot[cols] = (snapshot[cols] * 10000).round(0).astype(np.int64)
        
        
        display(snapshot.groupby("ID")["time"].min().max())
        
        
        
        snapshot = snapshot[["date", "time", "clockAtArrival", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQuantity", "totalAskQuantity","vwapBid", "vwapAsk",
        "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]]

In [2]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time
pd.set_option("max_columns", 200)

    
colnames=['StockID', 'DateTime', 'prevClose', 'openPrice','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','b1','b2','b3','b4','b5','b6','b7','b8','b9','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b20',
          'b21','b22','b23','b24','b25','b26','b27','b28','b29','b30','b31','b32','b33','b34','b35','b36','b37','b38','b39','b40','b41','b42','b43',
          'b44','b45','b46','b47','b48','b49','b50','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14',
         'a15','a16','a17','a18','a19','a20','a21','a22','a23','a24','a25','a26','a27','a28','a29','a30','a31','a32','a33','a34','a35','a36','a37','a38',
         'a39','a40','a41','a42','a43','a44','a45','a46','a47','a48','a49','a50','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames1=['StockID', 'DateTime', 'prevClose', 'openPrice','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n',
           'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
           'ask7n', 'ask8n', 'ask9n', 'ask10n','ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames2=['SecurityID', 'DateTime', 'Price', 'Qty', 'LeaveQty', "Side"]

startTm = datetime.datetime.now()
year = "2019"
df = []
bad = []

readPath = 'G:\\' + year +'\\***' 
# readPath = 'F:\\' + year +'\\***' 
dataPathLs = np.array(glob.glob(readPath))

snapshot = pd.DataFrame()

for data in dataPathLs:
    path = np.array(glob.glob(data +'\\***'))[[os.path.basename(i).split('.')[0] == 'Snapshot' \
                                                               for i in np.array(glob.glob(data +'\\***'))]][0]
    
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue

    else:
        try:
            snapshot = pd.read_csv(data +'\\Snapshot.csv', header=None)
            assert(snapshot.shape[1] == len(colnames))
            print("XTP lv2 data")
            snapshot.columns = colnames
            snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
            snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)
        except:
            try1 = pd.read_csv(data +'\\Snapshot.csv', nrows=1, header=None)
            if try1.shape[1] == 2:
                print("native lv2 data")
                snapshot = pd.read_csv(data +'\\Snapshot.csv', header=None, names=colnames1).iloc[1:, :]
                snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
                snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000000).astype(np.int64)  
                snapshot["StockID"] = snapshot["StockID"].astype(int)
                snapshot = snapshot[(snapshot["StockID"] >= 600000) & (snapshot["StockID"] < 700000)].reset_index(drop=True)
                print(int(snapshot["DateTime"].iloc[0]//1000000))
                print(snapshot.groupby("StockID")["time"].min().max())
                print(snapshot.groupby("StockID")["time"].min().min())


native lv2 data


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20190102
92553000000
92500000000
native lv2 data
20190103
92549000000
92500000000
native lv2 data
20190104
92530000000
92500000000
native lv2 data
20190107
92529000000
92500000000
native lv2 data
20190108
92530000000
92500000000
native lv2 data
20190109
92523000000
92500000000
native lv2 data
20190110
92537000000
92500000000
native lv2 data
20190111
92514000000
92500000000
native lv2 data
20190114
92549000000
92500000000
native lv2 data
20190115
92550000000
92500000000
native lv2 data
20190116
92540000000
92500000000
native lv2 data
20190117
92530000000
92500000000
native lv2 data
20190118
92533000000
92500000000
native lv2 data
20190121
92537000000
92500000000
native lv2 data
20190122
92538000000
92500000000
native lv2 data
20190123
92552000000
92500000000
native lv2 data
20190124
92530000000
92500000000
native lv2 data
20190125
92532000000
92500000000
native lv2 data
20190128
92534000000
92500000000
native lv2 data
20190129
92552000000
92500000000
native lv2 data
20190130
92534000000

In [59]:
pd.set_option("max_rows", 250)
snapshot.dtypes

date                      int64
time                      int64
clockAtArrival            int64
ID                        int32
ordering                  int64
cum_tradesCnt             int64
cum_volume                int64
cum_amount                int64
preClose                  int64
high                      int64
low                       int64
close                     int64
bid10p                    int64
bid9p                     int64
bid8p                     int64
bid7p                     int64
bid6p                     int64
bid5p                     int64
bid4p                     int64
bid3p                     int64
bid2p                     int64
bid1p                     int64
ask1p                     int64
ask2p                     int64
ask3p                     int64
ask4p                     int64
ask5p                     int64
ask6p                     int64
ask7p                     int64
ask8p                     int64
ask9p                     int64
ask10p  

In [60]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["ID"] == 1600000) & (snapshot["time"] > 93000000000)].head(20)

,date,time,clockAtArrival,ID,ordering,cum_tradesCnt,cum_volume,cum_amount,preClose,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid10n,bid9n,bid8n,bid7n,bid6n,bid5n,bid4n,bid3n,bid2n,bid1n,ask1n,ask2n,ask3n,ask4n,ask5n,ask6n,ask7n,ask8n,ask9n,ask10n,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,totalBidQuantity,totalAskQuantity,vwapBid,vwapAsk,totalBidOrders,totalAskOrders,totalBidLevels,totalAskLevels,bidTradeMaxDuration,askTradeMaxDuration,cum_canceledBuyOrders,cum_canceledBuyVolume,cum_canceledBuyAmount,cum_canceledSellOrders,cum_canceledSellVolume,cum_canceledSellAmount,cancelBuyOrders,cancelBuyVolume,cancelBuyAmount,cancelSellOrders,cancelSellVolume,cancelSellAmount
224132,20180103,93001000000,1514943001000000,1600000,205,43,46197,588111810000,127200,127400,127300,127400,126500,126600,126700,126800,126900,127000,127100,127200,127300,127400,127500,127600,127700,127800,127900,128000,128100,128200,128300,128400,45400,110200,69900,202700,8800,39700,121400,3200,23503,600,37600,31600,25500,44600,79900,563677,8700,108800,27100,16600,20,17,12,19,4,14,3,3,8,1,5,7,8,19,20,28,7,20,10,15,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,600,2000,5800,28800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1540103,2990518,125750,132360,401,847,72,116,0,1,13,22400,285678000000,4,53600,665172000000,0,0,0,0,0,0
225388,20180103,93004000000,1514943004000000,1600000,206,54,62897,800857810000,127200,127400,127300,127400,126500,126600,126700,126800,126900,127000,127100,127200,127300,127400,127500,127600,127700,127800,127900,128000,128100,128200,128300,128400,55200,113600,83700,204400,61200,46000,121400,3200,33403,14200,69300,37100,25500,47500,91900,573477,11100,110800,29800,17600,24,19,16,21,6,17,3,3,9,4,10,9,8,22,26,32,9,21,13,17,11600,1000,1500,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,600,2000,5800,28800,1400,1000,1000,20000,8300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1664303,3131218,125830,132240,435,909,73,117,0,1,14,23400,298378000000,4,53600,665172000000,1,1000,12700000000,0,0,0
226876,20180103,93007000000,1514943007000000,1600000,207,76,154800,1970925000000,127200,127400,127300,127300,126300,126400,126500,126600,126700,126800,126900,127000,127100,127200,127300,127400,127500,127600,127700,127800,127900,128000,128100,128200,11400,9700,70200,120600,87800,204400,61200,52400,124300,5300,10997,5100,69300,40600,26500,54900,94900,573977,11700,110800,9,7,26,21,19,21,6,19,5,4,6,1,10,10,9,26,28,33,10,21,2000,200,1000,2100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3897,100,4400,40

In [4]:
import datetime
datetime.datetime.fromtimestamp(1577928648000000/1e6).strftime("%Y-%m-%d %H:%M:%S %f")

'2020-01-02 09:30:48 000000'

In [5]:
snapshot.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\20180103_SH.pkl")

In [14]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

year = "2019"
df = []
bad = []
readPath = 'E:\\SZ\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
display(len(dataPathLs))
dataPathLs = np.delete(dataPathLs, np.argwhere(dataPathLs == 'E:\\SZ\\2019\\0109(深交所数据)')[0])
dataPathLs = np.delete(dataPathLs, np.argwhere(dataPathLs == 'E:\\SZ\\2019\\0314')[0])
dataPathLs = np.delete(dataPathLs, np.argwhere(dataPathLs == 'E:\\SZ\\2019\\0315(深交所数据)')[0])
display(len(dataPathLs))

232

229